In [1]:
import nibabel as nib
import os
from glob import glob
from natsort import natsorted
import numpy as np
from pprint import pprint

In [7]:
parc_paths = natsorted(glob(os.path.join("parcellation_data", "GIF_output", "vs_gk_*", "image_NeuroMorph_Parcellation.nii.gz")))
segm_paths = natsorted(glob(os.path.join("vs_manual_segmentations", "vs_gk_*", "vs_gk_seg_refT1.nii.gz")))

out_path = os.path.join("parcellation_data", "GIF_with_tumour_all_classes")

assert(len(parc_paths) == len(segm_paths)), f"Require same number of cases in both paths, but got {len(parc_paths)=}, {len(segm_paths)=}"

In [8]:
tumour_label_number = 300

for parc_path, segm_path in list(zip(parc_paths, segm_paths)):
    parc_nii = nib.load(parc_path)
    segm_nii = nib.load(segm_path)
    
    parc_data = np.array(parc_nii.get_fdata())
    segm_data = np.array(segm_nii.get_fdata())
    
    comb_data = segm_data.copy()
    comb_data[segm_data == 1] = tumour_label_number
    comb_data[segm_data != 1] = parc_data[segm_data != 1]

    # save new combined parcellation
    folder_name = parc_path.split(os.sep)[-2]
    outp_path = os.path.join(out_path, folder_name)
    
    os.makedirs(os.path.join(outp_path), exist_ok=True)
    
    comb_nii = nib.Nifti1Image(comb_data, parc_nii.affine)
    nib.save(comb_nii, os.path.join(outp_path, "parcellation_with_tumour.nii.gz"))